Orbit is a Python framework created by Uber for Bayesian time series forecasting and inference; it is built upon probabilistic programming packages like PyStan and Uber’s own Pyro. Orbit currently supports the implementations of the following forecasting models:

> * Exponential Smoothing (ETS)
> * Damped Local Trend (DLT)
> * Local Global Trend (LGT)

# Implementing Damp Local Trend models with Orbit

**DLT** stands for Damped Local Trend model, which is an important model type in orbit package. In the model equation, there is a local trend term and a global trend term.

In this notebook we will show how to use Orbit DLT models with the US unemployment claims data and different gloabl trend options

**Note: Negative response values are not allowed in dlt model, due to the existence of the global trend term.**

  Install orbit from PyPI



In [ ]:

!python -m pip install pip --upgrade --user -q --no-warn-script-location
!python -m pip install numpy pandas seaborn matplotlib scipy statsmodels sklearn tensorflow keras torch torchvision \
    tqdm scikit-image orbit-ml --user -q --no-warn-script-location
import IPython
IPython.Application.instance().kernel.do_shutdown(True)


In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta

from orbit.models.dlt import DLTMAP, DLTAggregated, DLTFull
from orbit.diagnostics.plot import plot_predicted_data
from orbit.diagnostics.plot import plot_predicted_components
from orbit.utils.dataset import load_iclaims

import warnings
warnings.filterwarnings('ignore')

## Data

*iclaims_example* is a dataset containing the weekly initial claims for US unemployment benefits against a few related google trend queries (unemploy, filling and job)from Jan 2010 - June 2018. 
This aims to mimick the dataset from the paper [Predicting the Present with Bayesian Structural Time Series](https://people.ischool.berkeley.edu/~hal/Papers/2013/pred-present-with-bsts.pdf) by SCOTT and VARIAN (2014).

Number of claims are obtained from [Federal Reserve Bank of St. Louis](https://fred.stlouisfed.org/series/ICNSA) while google queries are obtained through [Google Trends API](https://trends.google.com/trends/?geo=US).
Note that dataset is transformed by natural log before fitting in order to be fitted as a multiplicative model.

In [ ]:
# load data
df = load_iclaims()
date_col = 'week'
response_col = 'claims'
df.dtypes

In [ ]:
df.head(5)

### Train / Test Split

In [ ]:
test_size = 52
train_df = df[:-test_size]
test_df = df[-test_size:]

In [ ]:
train_df.head(5)

## DLT Model

**DLT** stands for Damped Local Trend model,

$$ y_{t}  = \hat{y}_t + \epsilon $$
$$\hat{y}_t=\mu_t + s_t + r_t $$
$$\mu_t=D(t) + l_{t-1} +  \theta{b_{t-1}}$$

with the update process as such 

$$g_t = D(t)$$
$$l_t = \rho_l(y_t - g_{t} - s_t - r_t) + (1-\rho_l)l_{t-1}$$
$$b_t=  \rho_b(l_t - l_{t-1}) + (1-\rho_b)\theta{b_{t-1}}$$
$$s_{t+m} =  \rho_s(y_t - l_t - r_t) + (1-\rho_s)s_t$$
$$r_t =  \Sigma_{j}\beta_j x_{jt}$$

There are a few choices of $D(t)$ as a global trend, such as linear, log-linear and logistic. Another feature of DLT is the regression component $r_t$. This serves the purpose of nowcasting or forecasting when exogenous regressors are known such as events and holidays.  Without losing generality, assume

$$\beta_j ~\sim \mathtt{Normal}(\mu_j, \sigma_j)$$

where $\mu_j = 0$  and $\sigma_j = 1$ by default as a non-informative prior. There are more choices of priors for the regression component in the package.

In orbit, we have three different wrappers for DLT models, `DLTMAP`, `DLTAggregated` and  `DLTFull`

### DLTMAP

DLT model for MAP (Maximum a Posteriori) prediction

In [ ]:
dlt = DLTMAP(
    response_col=response_col,
    date_col=date_col,
    seasonality=52,
    seed=8888,
)

In [ ]:
%%time
dlt.fit(df=train_df)

In [ ]:
predicted_df = dlt.predict(df=test_df)

In [ ]:
_ = plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df, 
                    date_col=date_col, actual_col=response_col,  
                    test_actual_df=test_df, title='Prediction with DLTMAP Model')

### DLTFull

LGT model for full prediction. In full prediction, the prediction occurs as a function of each parameter posterior sample, and the prediction results are aggregated after prediction. Prediction will always return the median (aka 50th percentile) along with any additional percentiles that are specified.

In [ ]:
dlt = DLTFull(
    response_col=response_col,
    date_col=date_col,
    seasonality=52,
    seed=8888
)

In [ ]:
%%time
dlt.fit(df=train_df)

In [ ]:
predicted_df = dlt.predict(df=test_df, store_prediction_array=True)

In [ ]:
dlt.predict

In [ ]:
predicted_df.tail(5)

In [ ]:
_ = plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df, 
                    date_col=dlt.date_col, actual_col=dlt.response_col, 
                    test_actual_df=test_df, title='Prediction with DLTFull Model')

### DLTAggregated

DLT model for aggregated posterior prediction. In aggregated prediction, the parameter posterior samples are reduced using `aggregate_method ({ 'mean', 'median' })` before performing a single prediction.

In [ ]:
dlt = DLTAggregated(
    response_col=response_col,
    date_col=date_col,
    seasonality=52,
    seed=8888
)

In [ ]:
%%time
dlt.fit(df=train_df)

In [ ]:
predicted_df = dlt.predict(df=test_df)

In [ ]:
predicted_df.tail(5)

In [ ]:
plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df, 
                    date_col=dlt.date_col, actual_col=dlt.response_col, 
                    test_actual_df=test_df, title='Prediction with DLTAggregated Model')

## DLT Model with Global Trend

The main differences between `DLT` and `LGT` are mainly:

- It introduces a damped factor on local trend
- It models global trend as a determinstic projection

There are four options in modeling the `global trend` with the `global_trend_option` arguemnt:

1. `linear`
2. `loglinear`
3. `logistic`
4. `flat`

The behavior of these three options will depend on the condition of `is_multiplicative` as well.  Below, we only show cases with condition when `is_multiplicative` as `True` since it is the default.

### Data

Instead of splitting the data into train and test. We will the full data set to train and simulate the multiplicative prediction data in the following 4 years:

In [ ]:
num_periods = 52 * 4
freq = 7
last_dt = (df[date_col].dt.to_pydatetime())[-1]
dts = [last_dt + timedelta(days=x * freq) for x in range(1, num_periods + 1)]
future_df = pd.DataFrame(dts, columns=[date_col])

### Linear Trend

In multiplicative model, the linear trend as default will be transformed as exponential growh/decay.

In [ ]:
dlt = DLTMAP(
    response_col=response_col,
    date_col=date_col,
    seasonality=52,
    seed=8888
)

In [ ]:
%%time
dlt.fit(df)

In [ ]:
predicted_df = dlt.predict(df=future_df, decompose=True)

In [ ]:
_ = plot_predicted_data(training_actual_df=df, predicted_df=predicted_df, 
                        date_col=dlt.date_col, actual_col=dlt.response_col,
                        title='DLT Model with Linear Trend')

### Log-Linear Trend

For users who want to maintain linear trend assumption in the multiplicative model, they can switch to `loglinear` global trend.  

In [ ]:
dlt_log = DLTMAP(
    response_col=response_col,
    date_col=date_col,
    seasonality=52,
    seed=8888,
    global_trend_option='loglinear'
)

In [ ]:
%%time
dlt_log.fit(df=df)

In [ ]:
predicted_df_dlt_log = dlt_log.predict(df=future_df, decompose=True)

In [ ]:
_ = plot_predicted_data(training_actual_df=df, predicted_df=predicted_df_dlt_log, 
                        date_col=dlt_log.date_col, actual_col=dlt_log.response_col,
                        title='DLT Model with Log-Linear Trend')

### Logistic Trend

In case of modeling logistic growth/decay, user can switch to `logistic` in the `global_trend_option`. 

In [ ]:
dlt_logit = DLTMAP(
    response_col=response_col,
    date_col=date_col,
    seasonality=52,
    seed=8888,
    global_trend_option='logistic'
)

In [ ]:
%%time
dlt_logit.fit(df=df)

In [ ]:
predicted_df_dlt_logit = dlt_logit.predict(df=future_df, decompose=True)

In [ ]:
_ = plot_predicted_data(training_actual_df=df, predicted_df=predicted_df_dlt_logit, 
                        date_col=dlt_log.date_col, actual_col=dlt_log.response_col,
                        title='DLT Model with Log-Linear Trend')

### Flat Trend

In [ ]:
dlt_logit = DLTMAP(
    response_col=response_col,
    date_col=date_col,
    seasonality=52,
    seed=8888,
    global_trend_option='flat'
)

In [ ]:
%%time
dlt_logit.fit(df=df)

In [ ]:
predicted_df_dlt_logit = dlt_logit.predict(df=future_df, decompose=True)

In [ ]:
_ = plot_predicted_data(training_actual_df=df, predicted_df=predicted_df_dlt_logit, 
                        date_col=dlt_log.date_col, actual_col=dlt_log.response_col,
                        title='DLT Model with Flat Trend')

More details for each method are available in the docstrings and also here: https://uber.github.io/orbit/orbit.models.html#module-orbit.models.dlt